In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StringType, StructType

In [2]:
spark = SparkSession.builder \
    .appName("KafkaIntegration") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.0") \
    .getOrCreate()
    #.config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.0") \
    #.getOrCreate()



:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/spark/.ivy2/cache
The jars for the packages stored in: /home/spark/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-996d0786-1451-4ca7-92a5-dad2bcfb538e;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central
	found org.apache.commons#commons-pool2;2.1

In [3]:
kafka_topic = "test-topic"
kafka_bootstrap_servers = "kafka:9092"  
message_schema = StructType().add("message", StringType())

In [4]:
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", kafka_topic) \
    .load()

In [5]:
df = df.selectExpr("CAST(value AS STRING)")


In [7]:
query = df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

23/12/02 22:21:48 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-22fbdb9a-0a7d-48ac-b1d1-b0951927b026. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/12/02 22:21:48 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+
|value|
+-----+
+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
-------------------------------------------
Batch: 1
-------------------------------------------
+----------------+
|           value|
+----------------+
|{ "data" : 111 }|
+----------------+

+----------------+
|           value|
+----------------+
|{ "data" : 111 }|
+----------------+



23/12/02 22:27:29 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 1001 (d162602abf23/192.168.64.8:9092) could not be established. Broker may not be available.
23/12/02 22:27:29 WARN NetworkClient: [AdminClient clientId=adminclient-2] Connection to node 1001 (d162602abf23/192.168.64.8:9092) could not be established. Broker may not be available.
23/12/02 22:27:29 WARN NetworkClient: [AdminClient clientId=adminclient-2] Connection to node 1001 (d162602abf23/192.168.64.8:9092) could not be established. Broker may not be available.
23/12/02 22:27:29 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 1001 (d162602abf23/192.168.64.8:9092) could not be established. Broker may not be available.
23/12/02 22:27:29 WARN NetworkClient: [AdminClient clientId=adminclient-2] Connection to node 1001 (d162602abf23/192.168.64.8:9092) could not be established. Broker may not be available.
23/12/02 22:27:29 WARN NetworkClient: [AdminClient clientId=admin